In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import math as mt
import os
import pickle
import mlflow
import mlflow.keras

In [3]:
# print("numpy=="+np.__version__)
# print("tensorflow=="+tf.__version__)
# print("pandas=="+pd.__version__)
# print("mlflow=="+mlflow.__version__)


AttributeError: module 'os' has no attribute '__version__'

In [2]:
bs=10
epochs=150

In [3]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1Metal device set to: Apple M1 Pro
 Physical GPUs, 1 Logical GPUs


2022-06-20 17:18:43.791220: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-06-20 17:18:43.791352: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)



systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



In [4]:



mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("graph-neural-network-trial-batch-norm")

<Experiment: artifact_location='./mlruns/4', experiment_id='4', lifecycle_stage='active', name='graph-neural-network-trial-batch-norm', tags={}>

In [5]:
mlflow.tensorflow.autolog()

In [ ]:
# def adjm(h,w,near):
#   am=[[0 for i in range(w*h)] for j in range(h*w)]
#   img=[[0 for i in range(w)] for j in range(h)]
  
#   dr=[[1,0],[0,1],[1,1],[-1,-1],[-1,0],[0,-1],[-1,1],[1,-1]]

#   for i,v in enumerate(img):
#     for j,v2 in enumerate(img[i]):
#       timg=[[0 for i in range(w)] for j in range(h)]
#       for k in range(0,near+1):
#         for d in dr:
#           posy=i+d[0]*k
#           posx=j+d[1]*k
#           if posy>=0 and posy<h and posx>=0 and posx<w:
#             timg[posy][posx]=1
#             am[i*w+j][posy*w+posx]=1
#         print(timg)  

#   return am

# val=adjm(4,5,1)

In [6]:
with open('data/train/x_train.pkl','rb') as f:
        x1=pickle.load(f)
with open('data/train/y_train.pkl','rb') as f:
        y1=pickle.load( f)
with open('data/valid/x_valid.pkl','rb') as f:
        x2=pickle.load(f)
with open('data/valid/y_valid.pkl','rb') as f:
        y2=pickle.load( f)
# with open('data/test/x_test.pkl','rb') as f:
#         x3=pickle.load(f)
# with open('data/test/y_test.pkl','rb') as f:
#         y3=pickle.load( f)

In [7]:
with open('data/test/x_test.pkl','rb') as f:
        x3=pickle.load(f)
with open('data/test/y_test.pkl','rb') as f:
        y3=pickle.load( f)

In [8]:
x1.shape

(420, 9, 12)

In [9]:
# print(max(y1[0]))

1.0


In [10]:
class GCN(tf.keras.layers.Layer):
    def __init__(self,inp_shape=9*12,out_shape=9*12,h=9, w=12,near=3):
        super(GCN, self).__init__()
        self.am=[[0 for i in range(w*h)] for j in range(h*w)]
        self.img=[[0 for i in range(w)] for j in range(h)]
  
        self.dr=[[1,0],[0,1],[1,1],[-1,-1],[-1,0],[0,-1],[-1,1],[1,-1]]

        for i,v in enumerate(self.img):
           for j,v2 in enumerate(self.img[i]):
        # timg=[[0 for i in range(w)] for j in range(h)]
              for k in range(0,near+1):
                for d in self.dr:
                  posy=i+d[0]*k
                  posx=j+d[1]*k
                  if posy>=0 and posy<h and posx>=0 and posx<w:
            # timg[posy][posx]=1
                    self.am[i*w+j][posy*w+posx]=1
        # print(timg) 
        self.am=np.array(self.am) 
        self.D = np.diag(np.sum(self.am, axis=0)) 
        self.Dinv=np.linalg.inv(self.D)
        self.am=tf.Variable(self.am,trainable=True,dtype=tf.float32)
        self.Dinv=tf.Variable(self.Dinv,trainable=True,dtype=tf.float32)
        # self.norm=tf.matmul(self.Dinv*self.am)
        self.w = self.add_weight(
            shape=(inp_shape, out_shape), initializer="random_normal", trainable=True,dtype=tf.float32
        )
        # self.b = self.add_weight(shape=(units,), initializer="zeros", trainable=True)

    def call(self, inputs):
        inputs=tf.cast(inputs,dtype=tf.float32)
    
        # print("inpshape",tf.transpose(inputs,perm=[1,0]).shape)
        # print(self.Dinv.shape)
        # print(self.am.shape)
        # print(self.w.shape)
        first_half=tf.matmul(inputs,tf.matmul(self.Dinv,self.am))
    
        return tf.matmul(first_half, self.w) 

In [24]:
def build_model(near=3,lay=2,bs=bs,n_out=7):
    inputs = tf.keras.Input( shape=(9,12),
    batch_size=bs)
    mlflow.log_param("layers", lay)
    mlflow.log_param("nearest", near)
    mlflow.set_tag("layer", "changed")
    print(inputs.shape)
    x=tf.reshape(inputs,[bs,inputs.shape[1]*inputs.shape[2]],name="initial reshape")
    for i in range(lay):
        x=GCN(near=near)(x)
        x=tf.keras.layers.Activation('relu',name="graph_layer"+str(i+1))(x)
    x=tf.keras.layers.BatchNormalization()(x)
    x=tf.keras.layers.Dense(units=32,activation="relu",name="dense1")(x)
    output=tf.keras.layers.Dense(n_out,activation="softmax",name="out_layer")(x)
    model = tf.keras.Model(inputs=inputs, outputs=output, name="graph_neuralnetwork")

    model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=["accuracy"],
)
    return model

In [25]:
model=build_model()

(10, 9, 12)


In [26]:
model.summary()

Model: "graph_neuralnetwork"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(10, 9, 12)]             0         
                                                                 
 tf.reshape_1 (TFOpLambda)   (10, 108)                 0         
                                                                 
 gcn_2 (GCN)                 (10, 108)                 34992     
                                                                 
 graph_layer1 (Activation)   (10, 108)                 0         
                                                                 
 gcn_3 (GCN)                 (10, 108)                 34992     
                                                                 
 graph_layer2 (Activation)   (10, 108)                 0         
                                                                 
 batch_normalization_1 (Batc  (10, 108)        

In [27]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=50)

In [24]:
# cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath="mode/base_model",
#                                                  save_weights_only=True,
#                                                  verbose=1)

In [28]:
# with mlflow.start_run(nested=true):
history = model.fit(x1,y1, batch_size=bs, epochs=epochs, callbacks=[callback],validation_data=(x2,y2),verbose=2)


Layer GCN has arguments ['self', 'inp_shape', 'out_shape', 'h', 'w', 'near']
in `__init__` and therefore must override `get_config()`.

Example:

class CustomLayer(keras.layers.Layer):
    def __init__(self, arg1, arg2):
        super().__init__()
        self.arg1 = arg1
        self.arg2 = arg2

    def get_config(self):
        config = super().get_config()
        config.update({
            "arg1": self.arg1,
            "arg2": self.arg2,
        })
        return config


2022/06/20 17:33:03 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during autologging: Changing param values is not allowed. Param with key='opt_learning_rate' was already logged with value='0.0001' for run ID='bc8b4c4cf7f94116bd09b91367fb4031'. Attempted logging new value '0.01'.

The cause of this error is typically due to repeated calls
to an individual run_id event logging.

Incorrect Example:
---------------------------------------
with mlflow.start_run():
    mlflow.log_param("depth", 3)
    mlflow.log_param("depth", 5)
---------------------------------------

Which will throw an MlflowException for overwriting a
logged parameter.

Correct Example:
---------------------------------------
with mlflow.start_run():
    with mlflow.start_run(nested=True):
        mlflow.log_param("depth", 3)
    with mlflow.start_run(nested=True):
        mlflow.log_param("depth", 5)
---------------------------------------

Which will create a new nested run for each individual


Epoch 1/150


2022-06-20 17:33:03.411341: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


42/42 - 1s - loss: 2.1005 - accuracy: 0.1357 - val_loss: 7.3416 - val_accuracy: 0.1500 - 748ms/epoch - 18ms/step
Epoch 2/150


2022-06-20 17:33:03.897058: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


42/42 - 0s - loss: 1.9110 - accuracy: 0.1690 - val_loss: 55.2584 - val_accuracy: 0.1929 - 421ms/epoch - 10ms/step
Epoch 3/150
42/42 - 0s - loss: 2.0709 - accuracy: 0.1881 - val_loss: 45.8470 - val_accuracy: 0.1429 - 424ms/epoch - 10ms/step
Epoch 4/150
42/42 - 0s - loss: 1.9398 - accuracy: 0.2214 - val_loss: 5.7503 - val_accuracy: 0.1357 - 406ms/epoch - 10ms/step
Epoch 5/150
42/42 - 0s - loss: 1.8807 - accuracy: 0.2024 - val_loss: 4.8632 - val_accuracy: 0.2214 - 403ms/epoch - 10ms/step
Epoch 6/150
42/42 - 0s - loss: 1.8779 - accuracy: 0.1976 - val_loss: 6.1984 - val_accuracy: 0.1429 - 408ms/epoch - 10ms/step
Epoch 7/150
42/42 - 0s - loss: 1.8221 - accuracy: 0.2333 - val_loss: 5.6577 - val_accuracy: 0.1429 - 399ms/epoch - 9ms/step
Epoch 8/150
42/42 - 0s - loss: 1.8137 - accuracy: 0.2429 - val_loss: 2.0452 - val_accuracy: 0.2214 - 401ms/epoch - 10ms/step
Epoch 9/150
42/42 - 0s - loss: 1.8076 - accuracy: 0.2548 - val_loss: 3.1856 - val_accuracy: 0.1500 - 414ms/epoch - 10ms/step
Epoch 10/15

2022/06/20 17:34:01 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during tensorflow autologging: 'NoneType' object has no attribute 'replace'


In [29]:
type(model)

keras.engine.functional.Functional